In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
df_product_silver=spark.read.format('delta').load('abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.product')

StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 5, Finished, Available, Finished)

In [4]:
df_region_silver=spark.read.format('delta').load('abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.region')

StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 6, Finished, Available, Finished)

In [5]:
df_sales_silver=spark.read.format('delta').load('abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/neha_karpe_silver_lakehouse.Lakehouse/Tables/dbo.sales')

StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 7, Finished, Available, Finished)

In [6]:
display(df_region_silver)

StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e90c8791-d7f8-409f-b179-71f355e3baad)

In [7]:
display(df_sales_silver)

StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, edfa9cba-e4df-432b-a020-32f7b5eb4bcc)

In [8]:
display(df_product_silver)

StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d9093ae1-431d-4489-b315-a3d8b6eb2678)

In [9]:
from pyspark.sql.functions import current_timestamp

# Create Product Dimension Table
df_product_dim = df_product_silver.select("ProductKey", "Product", "Standard_Cost", "Color", "Subcategory", "Category").dropDuplicates()

# Create Region Dimension Table
df_region_dim = df_region_silver.select("SalesTerritoryKey", "Region", "Country", "Group").dropDuplicates()

# Alias the DataFrames
sales = df_sales_silver.alias("sales")
product = df_product_dim.alias("product")
region = df_region_dim.alias("region")

# Join and create the Fact Table
df_sales_fact = sales \
    .join(product, sales["ProductKey"] == product["ProductKey"], "inner") \
    .join(region, sales["SalesTerritoryKey"] == region["SalesTerritoryKey"], "inner") \
    .select(
        sales["SalesOrderNumber"],
        sales["OrderDate"],
        sales["ProductKey"],
        sales["SalesTerritoryKey"],
        sales["OrderQuantity"],
        sales["UnitPrice"],
        sales["SalesAmount"],
        sales["TotalProductCost"]
    ) \
    .withColumn("LoadTimestamp", current_timestamp())


StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 11, Finished, Available, Finished)

In [10]:
# Define your Gold Lakehouse paths
gold_lakehouse_path = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/dummy_gold_lakehouse.Lakehouse/Tables"

gold_path_dim_product = f"{gold_lakehouse_path}/dim_product"
gold_path_dim_region = f"{gold_lakehouse_path}/dim_region"
gold_path_fact_sales = f"{gold_lakehouse_path}/fact_sales"

# Write Dimension: Product
df_product_dim.write.format("delta").mode("overwrite").save(gold_path_dim_product)

# Write Dimension: Region
df_region_dim.write.format("delta").mode("overwrite").save(gold_path_dim_region)

# Write Fact Table: Sales
df_sales_fact.write.format("delta").mode("overwrite").save(gold_path_fact_sales)

print("Gold layer tables written successfully to the Lakehouse.")



StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 12, Finished, Available, Finished)

Gold layer tables written successfully to the Lakehouse.


In [11]:
from datetime import datetime
import uuid

# Define Gold Lakehouse log file path (Files section)
log_path_gold = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/dummy_gold_lakehouse.Lakehouse/Files/gold_logs"

# Logging function
def log_gold_layer_load(table_name, record_count, log_path, status="success", operation="Gold Layer Load"):
    log_data = [{
        "log_id": str(uuid.uuid4()),
        "table_name": table_name,
        "record_count": record_count,
        "status": status,
        "timestamp": datetime.now().isoformat(),
        "operation": operation
    }]
    df_log = spark.createDataFrame(log_data)
    df_log.write.mode("append").json(log_path)
    print(f"Log written for {table_name}")

# Call log function for each table
log_gold_layer_load("dim_product", df_product_dim.count(), log_path_gold)
log_gold_layer_load("dim_region", df_region_dim.count(), log_path_gold)
log_gold_layer_load("fact_sales", df_sales_fact.count(), log_path_gold)


StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 13, Finished, Available, Finished)

Log written for dim_product
Log written for dim_region
Log written for fact_sales


In [12]:
from pyspark.sql.functions import sum, col

agg1 = df_sales_fact.alias("fact") \
    .join(df_region_dim.alias("region"), "SalesTerritoryKey") \
    .join(df_product_dim.alias("product"), "ProductKey") \
    .groupBy("Region", "Category") \
    .agg(
        sum("SalesAmount").alias("TotalSales"),
        sum("TotalProductCost").alias("TotalCost"),
        sum("OrderQuantity").alias("TotalQuantity")
    ) \
    .withColumn("Profit", col("TotalSales") - col("TotalCost"))


StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 14, Finished, Available, Finished)

In [14]:
from pyspark.sql.functions import col, sum, desc

top5_products = df_sales_fact.alias("fact") \
    .join(df_product_dim.alias("product"), "ProductKey") \
    .groupBy("ProductKey", "Product") \
    .agg(sum("SalesAmount").alias("TotalSales")) \
    .orderBy(desc("TotalSales")) \
    .limit(5)

top5_path = f"{gold_lakehouse_path}/dbo.agg_top_5_products"
top5_products.write.mode("overwrite").format("delta").save(top5_path)

print("Top 5 Products by Total Sales saved to Gold Lakehouse.")


StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 16, Finished, Available, Finished)

Top 5 Products by Total Sales saved to Gold Lakehouse.


In [16]:
from pyspark.sql.functions import year, month, sum

monthly_trend = df_sales_fact \
    .withColumn("Year", year("OrderDate")) \
    .withColumn("Month", month("OrderDate")) \
    .groupBy("Year", "Month") \
    .agg(sum("SalesAmount").alias("TotalSales")) \
    .orderBy("Year", "Month")

monthly_trend_path = f"{gold_lakehouse_path}/dbo.agg_monthly_sales_trend"
monthly_trend.write.mode("overwrite").format("delta").save(monthly_trend_path)

print("Monthly Sales Trend saved to Gold Lakehouse.")


StatementMeta(, f71678fa-762f-493c-9584-49bc7daac425, 18, Finished, Available, Finished)

Monthly Sales Trend saved to Gold Lakehouse.
